In [69]:
import urllib.request
from bs4 import BeautifulSoup
import requests
import pandas as pd

Read data from Wikipedia in xml form and find Table in variable 'table'

In [70]:
# getting data from internet
wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
raw_wikipedia_page= requests.get(wikipedia_link).text

# using beautiful soup to parse the HTML/XML codes.
soup = BeautifulSoup(raw_wikipedia_page,'xml')
#print(soup.prettify())
table = soup.find('table')

Retrive date from html table tags to seperate tables using for and Append

In [71]:
Postcode      = []
Borough       = []
Neighbourhood = []

for tr_cell in table.find_all('tr'):
    cells=tr_cell.findAll('td')
    if len(cells)==3:
# If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough.        
        if (cells[2].text == 'Not assigned\n'):
            cells[2].text == cells[1].text
# If borough is Not assigned then do not consider that record            
        if (cells[1].text == 'Not assigned\n'):
            continue
        Postcode.append(cells[0].text)
        Borough.append(cells[1].text)
        Neighbourhood.append(cells[2].text)

Create Pandas Data Frame

In [72]:
# define the dataframe columns
column_names = ['PostalCode','Borough', 'Neighborhood', 'Latitude', 'Longitude']  

# instantiate the dataframe
df_canada = pd.DataFrame(columns=column_names)

Assign Columns to Data Frame

In [73]:
df_canada['PostalCode']=Postcode
df_canada['Borough']=Borough
df_canada['Neighborhood']=Neighbourhood

Repleace '\n' in all 3 columns


In [75]:
df_canada['PostalCode']=df_canada['PostalCode'].str.replace('\n','')
df_canada['Borough']=df_canada['Borough'].str.replace('\n','')
df_canada['Neighborhood']=df_canada['Neighborhood'].str.replace('\n','')

In [76]:
df_canada.shape

(103, 5)

In [68]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
!pip install pgeocode
import pgeocode


In [77]:
# For each row update the 'Bonus' value to it's double
for index_label, row_series in df_canada.iterrows():
    nomi = pgeocode.Nominatim('CA')
    latitude = nomi.query_postal_code(row_series['PostalCode']).latitude
    longitude = nomi.query_postal_code(row_series['PostalCode']).longitude
    df_canada.at[index_label , 'Latitude'] =  latitude
    df_canada.at[index_label , 'Longitude'] =  longitude
  



In [78]:
df_canada

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7545,-79.33
1,M4A,North York,Victoria Village,43.7276,-79.3148
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.6518,-79.5076
99,M4Y,Downtown Toronto,Church and Wellesley,43.6656,-79.383
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.7804,-79.2505
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.6325,-79.4939


#### Create a map of New York with neighborhoods superimposed on top.


In [81]:
!pip install folium
import folium # map rendering library

In [106]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude


In [107]:
sub_string = 'Toronto'
toronto_data = df_canada[df_canada['Borough'].str.contains(sub_string)].reset_index(drop=True)


In [108]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for latitude, longitude, Borough, Neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(Neighborhood, Borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto